### Check similarity between stores 

In [29]:
# Create function to calculate correlation between trial store and control store 
def calculateCorrelation(store_list,measure_df,trial_store, measure_col):
    result = []
    for store in store_list:
        trial = measure_df.query('STORE_NBR == {}'.format(trial_store)).reset_index()
        control = measure_df.query('STORE_NBR == {}'.format(store)).reset_index()
        score = trial.corrwith(control)[measure_col]
        result.append({'store':store,'score':score})
    result = pd.DataFrame(result)
    #print(result.sort_values(by='score',ascending=False).head(5))
    return result 

In [32]:
# Create a function to calculate a standardised magnitude distance for a measure looping through each control store
def calculateMagnitudeDistance(store_list,measure_df,trial_store, measure_col):
    result = []
    for store in store_list:
        trial = measure_df.query('STORE_NBR == {}'.format(trial_store)).reset_index().drop(columns='STORE_NBR')
        control = measure_df.query('STORE_NBR == {}'.format(store)).reset_index().drop(columns='STORE_NBR')
        
        abs_diff = abs(trial[measure_col] - control[measure_col])
        min_measure = min(abs_diff)
        max_measure = max(abs_diff)
        magnitude_measure = 1 - (abs_diff - min_measure)/(max_measure - min_measure)
        score = magnitude_measure.mean()
        
        result.append({'store':store,'score':score})
    result = pd.DataFrame(result)
    #print(result.sort_values(by='score',ascending=False).head(5))
    return result 

In [33]:
def calculateFinalScore(score_sales,dis_sales,score_customer,dis_customer):
    # Create a combined score composed of correlation and magnitude
    corr_weight = 0.5
    mag_dist_weight = 0.5

    combine_sale_score = score_sales*corr_weight + dis_sales*mag_dist_weight
    combine_customer_score = score_customer*corr_weight + dis_customer*mag_dist_weight
    
    # Create final score by combine sales and customer score together 
    final_score = (combine_sale_score + combine_customer_score)/2
    
    # Final the store with highest score to be the control store
    control_store_id = final_score.sort_values(by='score', ascending=False).head(1)

    return control_store_id 

### Create new features 

In [4]:
def get_type(row, trial_store, control_store):
    if row['STORE_NBR'] == trial_store:
        return 'Trial'
    elif row['STORE_NBR'] == control_store:
        return 'Control'
    else:
        return 'Other' 

In [5]:
def createType(m_data,trial_store, control_store):
    # Create a new column type 
    flat = m_data.reset_index().query('y_m < "2019-02"')
    flat['Type'] = flat.apply(lambda x: get_type(x, trial_store,control_store),axis=1)
 
    return flat

In [14]:
def scaleAndCalculatePctDiff(pre_trial_df,whole_df, trial_store, control_store, col_name):

    
    # Scale pre‐trial control sales to match pre‐trial trial store sales
    if col_name == 'TOT_SALES':
        trial_sum = pre_trial_df.query('STORE_NBR == {}'.format(trial_store)).TOT_SALES.sum()
        control_sum = pre_trial_df.query('STORE_NBR == {}'.format(control_store)).TOT_SALES.sum()
        
    if col_name == 'LYLTY_CARD_NBR':
        trial_sum = pre_trial_df.query('STORE_NBR == {}'.format(trial_store)).LYLTY_CARD_NBR.nunique()
        control_sum = pre_trial_df.query('STORE_NBR == {}'.format(control_store)).LYLTY_CARD_NBR.nunique()
        
    scale_factor = trial_sum / control_sum
    
    #print(whole_df.reset_index().query('STORE_NBR == {}'.format(trial_store)))
    # Apply the scaling factor
    flat_trial_whole = whole_df.reset_index().query('STORE_NBR == {}'.format(trial_store)).drop(columns='STORE_NBR')
    flat_control_whole = whole_df.reset_index().query('STORE_NBR == {}'.format(control_store)).drop(columns='STORE_NBR')
    
    if col_name == 'TOT_SALES':
        merge_whole = pd.merge(flat_trial_whole,flat_control_whole,how='inner',on='y_m').rename(columns={'TOT_SALES_x':'Trial','TOT_SALES_y':'Control'})
    if col_name == 'LYLTY_CARD_NBR':
        merge_whole = pd.merge(flat_trial_whole,flat_control_whole,how='inner',on='y_m').rename(columns={'LYLTY_CARD_NBR_x':'Trial','LYLTY_CARD_NBR_y':'Control'})
        
    
    merge_whole['Control'] = merge_whole['Control']*scale_factor

    # Calculate the percentage difference between scaled control sales and trial sales
    merge_whole['pctDiff'] = abs((merge_whole['Control'] - merge_whole['Trial'])/merge_whole['Control'])
    
    return merge_whole

In [17]:
def add_95_5_data(trial_data, std_pre_trial, control_data, col_name):
    # Create new column Type 
    control_95 = control_data.copy()
    control_5 = control_data.copy()
      
    control_95[col_name] = control_data[col_name] * (1 + std_pre_trial*2)    
    control_5[col_name] = control_data[col_name] * (1 - std_pre_trial*2)
        
    control_95['Type'] = 'Control 95th % confidence interval'
    control_5['Type'] = 'Control 5th % confidenceinterval'
    
    trial_data['Type'] = 'Trial'
    control_data['Type'] = 'Control'

    list_df = [trial_data, control_data, control_5,control_95]
    df_with_conf_interval = pd.concat(list_df)
    
    return df_with_conf_interval

### Plot 

In [35]:
def plot_m_data(data,col_name,title):
    # Plot pre-trial period monthly sales for control, trial and other stores
    plt.style.use('seaborn')
    data.groupby(['Type','y_m']).mean()[col_name].swaplevel().unstack().plot(figsize=(10,5))
    plt.xlabel('Month of operation')
    plt.ylabel('Total {}'.format(title))
    plt.title('Avg monthly store {} by month over time'.format(title), fontsize = 15)
    plt.legend( ncol = 1, framealpha = 1);